# Obtención de datos desde archivos

## Objetivo
Breve descripción del objetivo (2–3 líneas)

## Fuentes de datos utilizadas
- CSV
- Excel
- Web (SII – Dólar o UF)

## Herramientas
- Python
- Pandas



## 1. Configuración inicial

En esta sección se importan las librerías necesarias para trabajar con los datos y se realizan configuraciones básicas del entorno.


In [283]:
import pandas as pd
import numpy as np


## 2. Creación de datos y carga de datos desde archivos


### 2.1 Generación de datos ficticios (fábrica de datos)

En esta sección se generan datos ficticios de clientes y transacciones mediante un proceso reproducible, con el objetivo de simular fuentes de datos reales utilizadas en un contexto de análisis empresarial.

La generación de estos datos permite contar con información estructurada sobre la cual aplicar posteriormente técnicas de carga, limpieza, transformación y exportación utilizando la biblioteca Pandas.

Los datos generados no representan información real, sino que fueron creados con fines académicos para ilustrar un flujo completo de trabajo en ciencia de datos.



In [284]:

rng = np.random.default_rng(42)

# -----------------------------
# 1) CLIENTES (similar al archivo)
# -----------------------------
n_clientes = 500

ids = np.arange(1, n_clientes + 1)

nombres_base = np.array([
    "Ana", "Sofía", "Camila", "Valentina", "Isidora", "Martina",
    "Mateo", "Benjamín", "Tomás", "Vicente", "Lucas", "Joaquín",
    "Daniela", "Fernanda", "Ignacio", "Gabriel", "Antonia", "Catalina"
])

apellidos_base = np.array([
    "González", "Muñoz", "Rojas", "Díaz", "Pérez", "Soto",
    "Contreras", "Silva", "Martínez", "Sepúlveda", "Torres", "Flores"
])

nombres = rng.choice(nombres_base, size=n_clientes) + " " + rng.choice(apellidos_base, size=n_clientes)

ciudades = np.array(["Santiago", "Valparaíso", "Concepción", "La Serena", "Antofagasta", "Temuco", "Rancagua"])
ciudad = rng.choice(ciudades, size=n_clientes, p=[0.45, 0.12, 0.14, 0.08, 0.08, 0.08, 0.05])

# Edad: distribución razonable para ecommerce (18–70)
edad = np.clip(rng.normal(loc=35, scale=12, size=n_clientes), 18, 70).round(0).astype(int)

# Total_Compras: más realista con Poisson (muchos con pocas compras, pocos con muchas)
total_compras = rng.poisson(lam=4.5, size=n_clientes)
total_compras = np.clip(total_compras, 0, 35)

# Ticket promedio (CLP) por cliente: lognormal para sesgo a la derecha (algunos gastan mucho)
ticket_promedio = rng.lognormal(mean=np.log(25000), sigma=0.55, size=n_clientes)

# Monto_Total = Total_Compras * Ticket_promedio + ruido
monto_total = (total_compras * ticket_promedio) + rng.normal(0, 5000, size=n_clientes)
monto_total = np.clip(monto_total, 0, None).round(0).astype(int)

clientes = pd.DataFrame({
    "ID": ids,
    "Nombre": nombres,
    "Edad": edad,
    "Ciudad": ciudad,
    "Total_Compras": total_compras,
    "Monto_Total": monto_total
})

# -----------------------------
# 2) TRANSACCIONES (opcional, pero recomendado)
# -----------------------------
# Genera una tabla de transacciones coherente con Total_Compras y Monto_Total.
# Ojo: si Total_Compras=0, ese cliente no tendrá transacciones.

n_transacciones = int(total_compras.sum())

# Repetimos el ID del cliente según su cantidad de compras
tx_id_cliente = np.repeat(ids, total_compras)

# Fechas aleatorias en los últimos 365 días
dias_atras = rng.integers(0, 365, size=n_transacciones)
tx_fecha = (pd.Timestamp.today().normalize() - pd.to_timedelta(dias_atras, unit="D")).astype("datetime64[ns]")

# Montos por transacción: repartimos el monto total del cliente en sus compras (con variación)
tx_monto = np.empty(n_transacciones, dtype=float)

start = 0
for i, c in enumerate(total_compras):
    if c == 0:
        continue
    end = start + c

    # pesos aleatorios para repartir (Dirichlet => suman 1)
    pesos = rng.dirichlet(alpha=np.ones(c) * 1.3)
    montos = pesos * monto_total[i]

    # ruido leve y mínimo razonable
    montos = np.clip(montos + rng.normal(0, 1500, size=c), 1500, None)
    tx_monto[start:end] = montos
    start = end

tx_monto = tx_monto.round(0).astype(int)

# Categorías ficticias
categorias = np.array(["Electrónica", "Hogar", "Moda", "Deportes", "Juguetes", "Belleza", "Alimentos"])
tx_categoria = rng.choice(categorias, size=n_transacciones, p=[0.18, 0.16, 0.20, 0.12, 0.10, 0.14, 0.10])

transacciones = pd.DataFrame({
    "ID_Transaccion": np.arange(1, n_transacciones + 1),
    "ID_Cliente": tx_id_cliente,
    "Fecha": tx_fecha,
    "Categoria": tx_categoria,
    "Monto": tx_monto
})



La generación de datos se ejecutó correctamente, creando los archivos correspondientes en el mismo directorio del notebook.  
Estos archivos serán utilizados a continuación para simular el proceso de carga de datos desde fuentes externas.


### 2.2 Exportación de datos a archivos CSV y Excel

In [285]:
# Exportación a CSV
df_clientes.to_csv("clientes_ficticios.csv", index=False, encoding="utf-8")
df_transacciones.to_csv("transacciones_ficticias.csv", index=False, encoding="utf-8")

# Exportación a Excel
df_clientes.to_excel("clientes_ficticios.xlsx", index=False)
df_transacciones.to_excel("transacciones_ficticias.xlsx", index=False)


En esta sección se exportan los conjuntos de datos generados a formatos de archivo estándar, específicamente CSV y Excel.
Esta exportación permite simular fuentes de datos externas, similares a las que se encuentran en entornos reales de trabajo.

El formato CSV se utiliza por su simplicidad y compatibilidad, mientras que el formato Excel facilita la visualización y revisión de los datos.
Los archivos generados serán utilizados en las siguientes secciones para la carga y procesamiento de información mediante Pandas.


### 2.3 Carga de datos desde archivo CSV

En esta sección se cargan los datos desde un archivo CSV utilizando la biblioteca Pandas.  
Este archivo contiene información estructurada que será utilizada como una de las fuentes principales del análisis.

El objetivo de este paso es verificar que los datos se importen correctamente y realizar una primera inspección del contenido, observando su estructura general, número de registros y columnas disponibles.


In [286]:
df_clientes = pd.read_csv("clientes_ficticios.csv")
df_transacciones = pd.read_csv("transacciones_ficticias.csv")


Los datos fueron cargados correctamente desde los archivos CSV.  
El conjunto de clientes contiene 500 registros, mientras que el conjunto de transacciones contiene 2221 registros.  
Esta información confirma que los datos están completos y listos para continuar con el proceso de limpieza y transformación.

In [287]:
df_clientes.shape


(500, 6)

El conjunto de datos contiene 500 registros correspondientes a clientes y 6 columnas con información demográfica y de comportamiento de compra. Esta estructura permite continuar con el proceso de análisis y limpieza de datos.


### 2.4 Carga de datos desde archivo xlsx

En esta sección se cargan los datos desde un archivo xlsx utilizando la biblioteca Pandas.  
Este archivo contiene información estructurada que será utilizada como una de las fuentes principales del análisis.

El objetivo de este paso es verificar que los datos se importen correctamente y realizar una primera inspección del contenido, observando su estructura general, número de registros y columnas disponibles.

In [288]:
df_clientes_excel = pd.read_excel("clientes_ficticios.xlsx")
df_clientes_excel.head()


,ID,Nombre,Edad,Ciudad,Total_Compras,Monto_Total
0,1,Sofía Muñoz,26,La Serena,3,86427
1,2,Fernanda Silva,46,Temuco,5,114303
2,3,Joaquín Flores,35,Temuco,3,43599
3,4,Benjamín Torres,32,Valparaíso,6,101303
4,5,Benjamín Díaz,34,Valparaíso,3,144745


In [289]:
df_clientes_excel.shape


(500, 6)

El conjunto de datos contiene 500 registros correspondientes a clientes y 6 columnas con información demográfica y de comportamiento de compra. Esta estructura permite continuar con el proceso de análisis y limpieza de datos.

### 2.5 Carga de datos desde tabla web (SII – Dólar)

En esta sección se extraen datos desde una tabla web utilizando la función `read_html()` de Pandas.  
La fuente corresponde al Servicio de Impuestos Internos (SII) de Chile, específicamente a los valores históricos del dólar.

El objetivo de este paso es demostrar la capacidad de Pandas para obtener datos directamente desde una página web y estructurarlos en un DataFrame para su posterior análisis.


In [290]:
url_dolar = "https://www.sii.cl/valores_y_fechas/dolar/dolar2026.htm"

tablas_dolar = pd.read_html(url_dolar)


In [291]:
len(tablas_dolar)


2

In [292]:
df_dolar = tablas_dolar[0]
df_dolar.head()


,0,1,2,3,4,5
0,1.0,NaN,11.0,NaN,21,889.54
1,2.0,907.13,12.0,895.54,22,878.21
2,3.0,NaN,13.0,885.71,23,872.22
3,4.0,NaN,14.0,883.96,24,NaN
4,5.0,901.55,15.0,882.74,25,NaN


La tabla presenta valores nulos debido a la estructura original del sitio web, lo cual es habitual en datos extraídos desde HTML.

In [293]:
df_dolar.shape


(11, 6)

La tabla web fue extraída correctamente desde el sitio del SII y convertida en un DataFrame.  
Los datos corresponden a valores históricos del dólar y se presentan de forma estructurada, permitiendo su análisis o uso como referencia externa.

En este caso, los datos no se integran directamente con los conjuntos de clientes y transacciones, ya que cumplen un rol informativo independiente dentro del proceso de obtención de datos.


## 3. Limpieza y estructuración de datos

Aquí trabajaremos con:

df_clientes
df_transacciones

### 3.0 Preparación de los datos (simulación de datos sucios)

Dado que los conjuntos de datos utilizados en este trabajo son de carácter ficticio y fueron generados sin inconsistencias, se introducen de forma intencional algunos problemas comunes en datos del mundo real, tales como valores nulos, registros duplicados y formatos inconsistentes.

El objetivo de esta etapa es simular escenarios reales de trabajo en ciencia de datos y permitir la aplicación de técnicas de limpieza y estructuración utilizando la biblioteca Pandas, manteniendo siempre una trazabilidad clara entre los datos originales y los datos modificados.


In [294]:
# Copias de trabajo para simular datos sucios
df_clientes_dirty = df_clientes.copy()
df_transacciones_dirty = df_transacciones.copy()


In [295]:
# Introducir valores nulos en la columna Edad (aprox. 3%)
idx_null_edad = df_clientes_dirty.sample(frac=0.03, random_state=42).index
df_clientes_dirty.loc[idx_null_edad, "Edad"] = np.nan


In [296]:
# Introducir duplicados de forma intencional (aprox. 2%)
duplicadas = df_transacciones_dirty.sample(frac=0.02, random_state=42)
df_transacciones_dirty = pd.concat(
    [df_transacciones_dirty, duplicadas],
    ignore_index=True
)


In [297]:
# Introducir formatos inconsistentes en Ciudad
idx_city = df_clientes_dirty.sample(frac=0.05, random_state=42).index
df_clientes_dirty.loc[idx_city, "Ciudad"] = (
    df_clientes_dirty.loc[idx_city, "Ciudad"]
    .str.upper()
    .str.strip()
)


### 3.1 Diagnóstico de valores nulos en clientes

En esta sección se realiza un diagnóstico inicial de los valores nulos presentes en el conjunto de datos de clientes.  
El objetivo es identificar columnas con información faltante y evaluar la magnitud del problema antes de tomar decisiones de limpieza o imputación.


In [298]:
df_clientes_dirty.isna().sum()
#Esto te devolverá:
#cuántos valores nulos que tiene cada columna

ID                0
Nombre            0
Edad             15
Ciudad            0
Total_Compras     0
Monto_Total       0
dtype: int64

In [299]:
df_clientes_dirty.isna().mean() * 100
#Esto te devolverá:
#cuántos valores el porcentaje de valores nulos que tiene cada columna

ID               0.0
Nombre           0.0
Edad             3.0
Ciudad           0.0
Total_Compras    0.0
Monto_Total      0.0
dtype: float64

El análisis de valores nulos en el conjunto de datos de clientes revela la presencia de información faltante en la columna **Edad**, con un total de 15 registros afectados. Estas ausencias fueron introducidas de forma intencional para simular escenarios reales de datos incompletos.

Dado que la proporción de valores nulos es baja en relación con el total de registros, es posible aplicar técnicas de imputación focalizadas sin comprometer significativamente la calidad ni la representatividad del conjunto de datos.



In [300]:
df_transacciones_dirty.isna().sum()
#Esto te mostrará:
#cada columna
#cuántos valores faltantes tiene

ID_Transaccion    0
ID_Cliente        0
Fecha             0
Categoria         0
Monto             0
dtype: int64

Tras analizar el conjunto de datos de transacciones, no se identifican valores nulos en ninguna de sus columnas.
Esto indica que la información se encuentra completa para las variables consideradas, por lo que no es necesario aplicar técnicas de imputación o eliminación de registros en esta etapa.

## 3.2 Tratamiento de valores nulos


Dado que la columna **Edad** presenta un número reducido de valores nulos (aproximadamente un 3% del total de registros), se opta por aplicar una estrategia de imputación en lugar de eliminar las filas afectadas.

Para este caso, se utiliza la **mediana** de la columna Edad como valor de imputación, ya que es una medida robusta frente a posibles valores extremos y permite conservar la distribución general de la variable sin introducir sesgos significativos.


In [301]:
# Calcular la mediana de la edad (ignorando valores nulos)
mediana_edad = df_clientes_dirty["Edad"].median()
mediana_edad


35.0

In [302]:
# Imputar valores nulos en Edad con la mediana
df_clientes_dirty["Edad"] = df_clientes_dirty["Edad"].fillna(mediana_edad)


In [303]:
df_clientes_dirty.isna().sum()


ID               0
Nombre           0
Edad             0
Ciudad           0
Total_Compras    0
Monto_Total      0
dtype: int64

Tras aplicar la imputación mediante la mediana, el conjunto de datos de clientes ya no presenta valores nulos en ninguna de sus columnas. Esta decisión permite conservar la totalidad de los registros y asegura la continuidad del análisis sin pérdida significativa de información.


## 3.3 Detección y tratamiento de registros duplicados

En esta etapa se analiza la presencia de registros duplicados en los conjuntos de datos. Los duplicados pueden generar distorsiones en los análisis, especialmente en métricas agregadas como conteos, promedios o sumas, por lo que es fundamental identificarlos y tratarlos adecuadamente.

El análisis se centra principalmente en el conjunto de datos de transacciones, donde la duplicación de registros es un problema común en sistemas de registro y consolidación de información.


In [304]:
# Identificar registros duplicados en transacciones
df_transacciones_dirty.duplicated().sum()


np.int64(44)

In [305]:
# Visualizar algunos registros duplicados
df_transacciones_dirty[df_transacciones_dirty.duplicated()].head()


,ID_Transaccion,ID_Cliente,Fecha,Categoria,Monto
2221,97,21,2025-07-20,Belleza,29723
2222,1500,340,2026-01-06,Deportes,32704
2223,1941,435,2025-06-04,Juguetes,2823
2224,1873,418,2025-11-11,Deportes,22043
2225,701,159,2025-08-25,Belleza,1500


Dado que los registros duplicados representan repeticiones exactas de transacciones ya registradas, se opta por eliminar dichos duplicados conservando una única ocurrencia de cada registro. Esta decisión permite mantener la integridad del conjunto de datos y evitar la sobreestimación de métricas en análisis posteriores.


In [306]:
# Eliminar registros duplicados
df_transacciones_dirty = df_transacciones_dirty.drop_duplicates()


In [307]:
# Identificar registros duplicados en transacciones 
df_transacciones_dirty.duplicated().sum()

np.int64(0)

In [308]:
# Visualizar algunos registros duplicados
df_transacciones_dirty[df_transacciones_dirty.duplicated()].head()

,ID_Transaccion,ID_Cliente,Fecha,Categoria,Monto


Se aprecia que transacciones_dirty ahora está limpio

### 3.4 Normalización de formatos (Ciudad)

En esta etapa se normaliza el formato de la variable categórica **Ciudad**, ya que en escenarios reales es común encontrar inconsistencias como:
- Uso de mayúsculas/minúsculas mezcladas (ej.: "TEMUCO" vs "Temuco")
- Espacios adicionales al inicio o al final (ej.: "Santiago ")

Estas inconsistencias pueden generar categorías duplicadas y afectar análisis posteriores (por ejemplo, conteos por ciudad o agrupaciones con `groupby`).  
Por ello, se aplica una estandarización básica del texto para asegurar consistencia.


In [309]:
# Revisar valores únicos y frecuencia (antes de normalizar)
df_clientes_dirty["Ciudad"].value_counts().head(15)



Ciudad
Santiago       219
Valparaíso      59
Concepción      54
Antofagasta     42
La Serena       37
Temuco          35
Rancagua        29
SANTIAGO         9
LA SERENA        6
TEMUCO           4
CONCEPCIÓN       2
RANCAGUA         2
VALPARAÍSO       2
Name: count, dtype: int64

In [310]:
# Cantidad de categorías distintas
df_clientes_dirty["Ciudad"].nunique()


13

In [311]:
# Normalizar Ciudad: quitar espacios y estandarizar formato
df_clientes_dirty["Ciudad"] = (
    df_clientes_dirty["Ciudad"]
    .astype(str)        # por seguridad si hay None/NaN en algún momento
    .str.strip()        # elimina espacios al inicio y fin
    .str.title()        # "SANTIAGO" -> "Santiago", "valparaíso" -> "Valparaíso"
)


In [312]:
# Revisar frecuencia (después de normalizar)
df_clientes_dirty["Ciudad"].value_counts().head(15)


Ciudad
Santiago       228
Valparaíso      61
Concepción      56
La Serena       43
Antofagasta     42
Temuco          39
Rancagua        31
Name: count, dtype: int64

In [313]:
# Ver cuántas categorías quedan después
df_clientes_dirty["Ciudad"].nunique()


7

Tras la normalización, la columna **Ciudad** queda estandarizada en un formato consistente, reduciendo el riesgo de categorías duplicadas por diferencias de escritura.  
Esto mejora la calidad del dataset y asegura resultados más confiables en análisis y agrupaciones posteriores.


In [314]:
df_clientes_dirty["Ciudad"].unique()[:20]


array(['La Serena', 'Temuco', 'Valparaíso', 'Antofagasta', 'Santiago',
       'Concepción', 'Rancagua'], dtype=object)

### 3.5 Verificación y ajuste de tipos de datos



En esta etapa se revisan los tipos de datos de cada columna en los conjuntos de clientes y transacciones, con el objetivo de asegurar que las variables se encuentren correctamente tipadas según su naturaleza.

Una correcta definición de los tipos de datos es fundamental para:
- Evitar errores en cálculos y transformaciones posteriores.
- Optimizar el uso de memoria.
- Facilitar análisis estadísticos y agregaciones.


3.5.1 Revisión de tipos actuales

In [315]:
# Tipos de datos en clientes
df_clientes_dirty.dtypes


ID                 int64
Nombre            object
Edad             float64
Ciudad            object
Total_Compras      int64
Monto_Total        int64
dtype: object

In [316]:
# Tipos de datos en transacciones
df_transacciones_dirty.dtypes


ID_Transaccion     int64
ID_Cliente         int64
Fecha             object
Categoria         object
Monto              int64
dtype: object

3.5.2 Ajuste de tipos de datos

In [317]:
# Asegurar que Edad sea numérica entera
df_clientes_dirty["Edad"] = df_clientes_dirty["Edad"].astype(int)

# Asegurar que Monto_Total y Total_Compras sean numéricos
df_clientes_dirty["Monto_Total"] = df_clientes_dirty["Monto_Total"].astype(int)
df_clientes_dirty["Total_Compras"] = df_clientes_dirty["Total_Compras"].astype(int)

# Convertir Fecha a formato datetime en transacciones
df_transacciones_dirty["Fecha"] = pd.to_datetime(df_transacciones_dirty["Fecha"])


3.5.3 Verificación posterior

In [318]:
df_clientes_dirty.dtypes


ID                int64
Nombre           object
Edad              int64
Ciudad           object
Total_Compras     int64
Monto_Total       int64
dtype: object

In [319]:
df_transacciones_dirty.dtypes


ID_Transaccion             int64
ID_Cliente                 int64
Fecha             datetime64[ns]
Categoria                 object
Monto                      int64
dtype: object

Tras la verificación y ajuste de los tipos de datos, los conjuntos de clientes y transacciones presentan una estructura consistente y adecuada para su análisis.

Las variables numéricas, categóricas y temporales se encuentran correctamente definidas, lo que permite avanzar con seguridad hacia las etapas de transformación y optimización de los datos.


## 4. Transformación y optimización de datos


### 4.1 Selección de columnas relevantes

En esta sección se realizan transformaciones orientadas a mejorar la estructura, legibilidad y utilidad de los conjuntos de datos para su análisis posterior.  
Estas transformaciones no modifican el contenido esencial de la información, sino que optimizan su presentación y organización.

In [320]:
# Selección de columnas relevantes en clientes
df_clientes_final = df_clientes_dirty[
    ["ID", "Nombre", "Edad", "Ciudad", "Total_Compras", "Monto_Total"]
]

# Selección de columnas relevantes en transacciones
df_transacciones_final = df_transacciones_dirty[
    ["ID_Transaccion", "ID_Cliente", "Fecha", "Categoria", "Monto"]
]


### 4.2 Renombrado de columnas para mayor legibilidad

Con el fin de mejorar la claridad y consistencia de los nombres de las variables, se renombran las columnas utilizando un formato uniforme en minúsculas y con guiones bajos.


In [321]:
# Renombrar columnas de clientes
df_clientes_final = df_clientes_final.rename(columns={
    "ID": "id_cliente",
    "Nombre": "nombre",
    "Edad": "edad",
    "Ciudad": "ciudad",
    "Total_Compras": "total_compras",
    "Monto_Total": "monto_total"
})

# Renombrar columnas de transacciones
df_transacciones_final = df_transacciones_final.rename(columns={
    "ID_Transaccion": "id_transaccion",
    "ID_Cliente": "id_cliente",
    "Fecha": "fecha",
    "Categoria": "categoria",
    "Monto": "monto"
})


### 4.3 Ordenamiento de los datos

Para facilitar la lectura y el análisis cronológico, se ordenan los conjuntos de datos utilizando columnas clave.


In [322]:
# Ordenar clientes por id_cliente
df_clientes_final = df_clientes_final.sort_values(by="id_cliente")

# Ordenar transacciones por fecha
df_transacciones_final = df_transacciones_final.sort_values(by="fecha")


### 4.4 Verificación del resultado final

In [323]:
df_clientes_final.head()


,id_cliente,nombre,edad,ciudad,total_compras,monto_total
0,1,Sofía Muñoz,26,La Serena,3,86427
1,2,Fernanda Silva,46,Temuco,5,114303
2,3,Joaquín Flores,35,Temuco,3,43599
3,4,Benjamín Torres,32,Valparaíso,6,101303
4,5,Benjamín Díaz,34,Valparaíso,3,144745


In [324]:
df_transacciones_final.head()


,id_transaccion,id_cliente,fecha,categoria,monto
103,104,23,2025-01-29,Moda,18901
332,333,71,2025-01-29,Belleza,34226
449,450,99,2025-01-29,Moda,54713
1068,1069,235,2025-01-29,Electrónica,49302
938,939,209,2025-01-30,Moda,13983


In [325]:
df_clientes_final.shape, df_transacciones_final.shape


((500, 6), (2221, 5))

Tras aplicar las transformaciones realizadas, los conjuntos de datos presentan una estructura clara, consistente y optimizada.  
Las columnas relevantes se encuentran correctamente nombradas y ordenadas, lo que facilita su integración y análisis en etapas posteriores.


## 5. Exportación de datos procesados

5.0 Objetivo

En esta sección se exportan los datos finales, una vez aplicadas las etapas de limpieza y estructuración.
El objetivo es generar un dataset listo para análisis posterior, guardándolo en formatos estándar (CSV y Excel) sin incluir el índice del DataFrame.


### 5.1 Definir el DataFrame final (Code)

In [326]:
df_clientes_final = df_clientes_dirty.copy()
df_transacciones_final = df_transacciones_dirty.copy()


### 5.2 Exportar a CSV (sin índice)

In [327]:
df_clientes_final.to_csv("clientes_limpio.csv", index=False, encoding="utf-8")
df_transacciones_final.to_csv("transacciones_limpio.csv", index=False, encoding="utf-8")


### 5.3 Exportar a Excel

In [328]:
df_clientes_final.to_excel("clientes_limpio.xlsx", index=False)
df_transacciones_final.to_excel("transacciones_limpio.xlsx", index=False)


### 5.4 Verificación rápida (Code)

In [329]:
pd.read_csv("clientes_limpio.csv").head()


,ID,Nombre,Edad,Ciudad,Total_Compras,Monto_Total
0,1,Sofía Muñoz,26,La Serena,3,86427
1,2,Fernanda Silva,46,Temuco,5,114303
2,3,Joaquín Flores,35,Temuco,3,43599
3,4,Benjamín Torres,32,Valparaíso,6,101303
4,5,Benjamín Díaz,34,Valparaíso,3,144745


### 5.5 “Antes vs Después”

In [330]:
print("ANTES (clientes):")
display(df_clientes.head())

print("DESPUÉS (clientes final):")
display(df_clientes_final.head())


ANTES (clientes):


,ID,Nombre,Edad,Ciudad,Total_Compras,Monto_Total
0,1,Sofía Muñoz,26,La Serena,3,86427
1,2,Fernanda Silva,46,Temuco,5,114303
2,3,Joaquín Flores,35,Temuco,3,43599
3,4,Benjamín Torres,32,Valparaíso,6,101303
4,5,Benjamín Díaz,34,Valparaíso,3,144745


DESPUÉS (clientes final):


,ID,Nombre,Edad,Ciudad,Total_Compras,Monto_Total
0,1,Sofía Muñoz,26,La Serena,3,86427
1,2,Fernanda Silva,46,Temuco,5,114303
2,3,Joaquín Flores,35,Temuco,3,43599
3,4,Benjamín Torres,32,Valparaíso,6,101303
4,5,Benjamín Díaz,34,Valparaíso,3,144745


## 6. Conclusiones


## Conclusiones

En este trabajo se desarrolló un flujo completo de obtención, limpieza y estructuración de datos utilizando la biblioteca Pandas, simulando un escenario real de trabajo en ciencia de datos.

A partir de datos ficticios generados de forma controlada, se realizó la exportación y posterior carga desde distintos formatos (CSV, Excel y tabla web), permitiendo reproducir problemáticas comunes como valores nulos, registros duplicados y formatos inconsistentes.

La etapa de limpieza fue clave para garantizar la calidad de la información, aplicando técnicas de imputación, eliminación de duplicados y normalización de datos, lo que permitió obtener conjuntos de datos coherentes y listos para análisis posterior.

Este ejercicio demuestra la importancia de contar con procesos estructurados y reproducibles en la obtención de datos, ya que reduce errores, mejora la trazabilidad y facilita el análisis confiable de la información en contextos reales de negocio.
